## Группа DS03-onl

Студент Парфимович Алексей

## Домашнее задание №22

Рекомендательные системы.

Изучить работу алгоритма Apriori на примере приложенного набора данных.  
Рассмотреть различные реализации алгоритма

In [59]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.metrics import mean_squared_error

%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (18, 5)

RANDOM_STATE = 42

Загрузить данные.

In [60]:
# Файл данных не содержит строки заголовков - отключим получение заголовоков из файла
df = pd.read_csv('dataset.csv', delimiter=',', header=None, skipinitialspace=True)

# Заменить значения NAN на пустую строку ''
df.fillna('', inplace=True)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,pork,sandwich bags,lunch meat,all- purpose,flour,soda,butter,vegetables,beef,aluminum foil,all- purpose,dinner rolls,shampoo,all- purpose
1,shampoo,hand soap,waffles,vegetables,cheeses,mixes,milk,sandwich bags,laundry detergent,dishwashing liquid/detergent,waffles,individual meals,hand soap,vegetables
2,pork,soap,ice cream,toilet paper,dinner rolls,hand soap,spaghetti sauce,milk,ketchup,sandwich loaves,poultry,toilet paper,ice cream,ketchup
3,juice,lunch meat,soda,toilet paper,all- purpose,,,,,,,,,
4,pasta,tortillas,mixes,hand soap,toilet paper,vegetables,vegetables,paper towels,vegetables,flour,vegetables,pork,poultry,eggs


In [61]:
# Итерация по строкам набора данных (rows):
# - отобрать в список уникальные и не пустые значения из строки набора  
basket = []
for i, row in df.iterrows():
    basket.append(row.loc[row != ''].unique().tolist())

basket

[['pork',
  'sandwich bags',
  'lunch meat',
  'all- purpose',
  'flour',
  'soda',
  'butter',
  'vegetables',
  'beef',
  'aluminum foil',
  'dinner rolls',
  'shampoo'],
 ['shampoo',
  'hand soap',
  'waffles',
  'vegetables',
  'cheeses',
  'mixes',
  'milk',
  'sandwich bags',
  'laundry detergent',
  'dishwashing liquid/detergent',
  'individual meals'],
 ['pork',
  'soap',
  'ice cream',
  'toilet paper',
  'dinner rolls',
  'hand soap',
  'spaghetti sauce',
  'milk',
  'ketchup',
  'sandwich loaves',
  'poultry'],
 ['juice', 'lunch meat', 'soda', 'toilet paper', 'all- purpose'],
 ['pasta',
  'tortillas',
  'mixes',
  'hand soap',
  'toilet paper',
  'vegetables',
  'paper towels',
  'flour',
  'pork',
  'poultry',
  'eggs'],
 ['toilet paper',
  'eggs',
  'vegetables',
  'bagels',
  'dishwashing liquid/detergent',
  'cereals',
  'paper towels',
  'laundry detergent',
  'butter',
  'shampoo'],
 ['paper towels',
  'tortillas',
  'vegetables',
  'milk',
  'ice cream',
  'juice',
  

Подготовить данные для обучения.

In [62]:
min_len = 1
max_len = 3
min_threshold = 1.5
min_confidence = 0.2
min_support = 7/len(basket)

### Реализация алгоритма Apriori от mlxtend

In [72]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder


encoder = TransactionEncoder()
transactions = pd.DataFrame(encoder.fit(basket).transform(basket), columns=encoder.columns_)

transactions.head()

,all- purpose,aluminum foil,bagels,beef,butter,cereals,cheeses,coffee/tea,dinner rolls,dishwashing liquid/detergent,...,shampoo,soap,soda,spaghetti sauce,sugar,toilet paper,tortillas,vegetables,waffles,yogurt
0,True,True,False,True,True,False,False,False,True,False,...,True,False,True,False,False,False,False,True,False,False
1,False,False,False,False,False,False,True,False,False,True,...,True,False,False,False,False,False,False,True,True,False
2,False,False,False,False,False,False,False,False,True,False,...,False,True,False,True,False,True,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,True,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,True,True,False,False


In [73]:
results = apriori(transactions, min_support= min_support, use_colnames=True, max_len=max_len)
results

,support,itemsets
0,0.263509,(all- purpose)
1,0.264176,(aluminum foil)
2,0.278185,(bagels)
3,0.262842,(beef)
4,0.261508,(butter)
...,...,...
9172,0.030020,"(yogurt, toilet paper, waffles)"
9173,0.036024,"(vegetables, waffles, tortillas)"
9174,0.045364,"(yogurt, vegetables, tortillas)"
9175,0.014676,"(yogurt, waffles, tortillas)"


In [74]:
rules = association_rules(results, metric="lift",  min_threshold = min_threshold)
rules.nlargest(n = 10, columns = "lift")

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
16,"(sandwich loaves, yogurt)",(juice),0.064043,0.257505,0.026017,0.406250,1.577639,0.009526,1.250518
17,(juice),"(sandwich loaves, yogurt)",0.257505,0.064043,0.026017,0.101036,1.577639,0.009526,1.041151
8,"(dishwashing liquid/detergent, spaghetti sauce)",(fruits),0.061374,0.263509,0.025350,0.413043,1.567474,0.009178,1.254762
9,(fruits),"(dishwashing liquid/detergent, spaghetti sauce)",0.263509,0.061374,0.025350,0.096203,1.567474,0.009178,1.038535
4,"(beef, mixes)",(pork),0.065377,0.250167,0.025350,0.387755,1.549986,0.008995,1.224728
5,(pork),"(beef, mixes)",0.250167,0.065377,0.025350,0.101333,1.549986,0.008995,1.040011
15,(juice),"(sandwich loaves, pork)",0.257505,0.060707,0.024016,0.093264,1.536298,0.008384,1.035906
14,"(sandwich loaves, pork)",(juice),0.060707,0.257505,0.024016,0.395604,1.536298,0.008384,1.228492
12,"(mixes, juice)",(ketchup),0.071381,0.250167,0.027352,0.383178,1.531688,0.009494,1.215639
13,(ketchup),"(mixes, juice)",0.250167,0.071381,0.027352,0.109333,1.531688,0.009494,1.042611


### Реализация алгоритма Apriori от apyori

In [75]:
from apyori import apriori
rules = apriori(transactions = basket, min_support = min_support, min_confidence = min_confidence, min_lift = min_threshold, min_length = min_len, max_length = max_len)
results = list(rules)

In [76]:
def inspect(results):
    ants = [tuple(result[2][0][0])[0] for result in results]
    cons = [tuple(result[2][0][1])[0] for result in results]
    supports = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts = [result[2][0][3] for result in results]
    return list (zip(ants, cons, supports, confidences, lifts))
    
resultsinDataFrame = pd.DataFrame(inspect(results), columns = ["antecedents", "consequents", "support", "confidence", "lift"])

In [77]:
resultsinDataFrame.nlargest(n = 10, columns = "lift")

,antecedents,consequents,support,confidence,lift
8,sandwich loaves,juice,0.026017,0.406250,1.577639
4,dishwashing liquid/detergent,fruits,0.025350,0.413043,1.567474
2,beef,pork,0.025350,0.387755,1.549986
7,sandwich loaves,juice,0.024016,0.395604,1.536298
6,mixes,ketchup,0.027352,0.383178,1.531688
9,laundry detergent,ketchup,0.022682,0.382022,1.527071
5,toilet paper,soap,0.025350,0.404255,1.522560
1,cheeses,sandwich bags,0.025350,0.380000,1.518987
3,cheeses,soap,0.026017,0.402062,1.514298
0,tortillas,sandwich loaves,0.022015,0.375000,1.507038


### Реализация алгоритма FP-Growth

In [78]:
from mlxtend.frequent_patterns import fpgrowth

results = fpgrowth(transactions, min_support= min_support, use_colnames=True, max_len=max_len)
results

,support,itemsets
0,0.597065,(vegetables)
1,0.276184,(lunch meat)
2,0.274183,(soda)
3,0.264176,(aluminum foil)
4,0.263509,(all- purpose)
...,...,...
9172,0.023349,"(fruits, poultry, mixes)"
9173,0.018012,"(fruits, poultry, bagels)"
9174,0.026017,"(dishwashing liquid/detergent, fruits, poultry)"
9175,0.026017,"(fruits, cereals, poultry)"


In [79]:
rules = association_rules(results, metric="lift",  min_threshold = min_threshold)
rules.nlargest(n = 10, columns = "lift")

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
14,"(sandwich loaves, yogurt)",(juice),0.064043,0.257505,0.026017,0.406250,1.577639,0.009526,1.250518
15,(juice),"(sandwich loaves, yogurt)",0.257505,0.064043,0.026017,0.101036,1.577639,0.009526,1.041151
6,"(dishwashing liquid/detergent, spaghetti sauce)",(fruits),0.061374,0.263509,0.025350,0.413043,1.567474,0.009178,1.254762
7,(fruits),"(dishwashing liquid/detergent, spaghetti sauce)",0.263509,0.061374,0.025350,0.096203,1.567474,0.009178,1.038535
4,"(beef, mixes)",(pork),0.065377,0.250167,0.025350,0.387755,1.549986,0.008995,1.224728
5,(pork),"(beef, mixes)",0.250167,0.065377,0.025350,0.101333,1.549986,0.008995,1.040011
11,(juice),"(sandwich loaves, pork)",0.257505,0.060707,0.024016,0.093264,1.536298,0.008384,1.035906
10,"(sandwich loaves, pork)",(juice),0.060707,0.257505,0.024016,0.395604,1.536298,0.008384,1.228492
8,"(mixes, juice)",(ketchup),0.071381,0.250167,0.027352,0.383178,1.531688,0.009494,1.215639
9,(ketchup),"(mixes, juice)",0.250167,0.071381,0.027352,0.109333,1.531688,0.009494,1.042611


### Выводы:
- значения метрик работы различных реализаций и правда не отличаются
- алгоритм fp-growth показывает лучшую производительность